In [1]:
import pandas as pd
import numpy as np 
from copy import deepcopy
from util.dataloader import cached_bison_data, load_pump_curves, cached_site_info, fetch_bison_data
from util.preprocessing import *
from util.calculations import *
from util.format_calculations import *
import plotly.express as px

# fetch data
# filepath = "/Users/audreyder/Neuralix/syncdatabase_2025-02-20.csv"
# data = fetch_bison_data(filepath)

# load data
filepath = "/Users/audreyder/Neuralix/syncdatabase_2025-02-20.csv"
data = cached_bison_data(filepath) 
# filter data / preprocess
data = threshold_filtering(data) 
sites_info = cached_site_info(dict=True)
sites_calib = [33404, 33467, 57740, 33614]
sites_nocalib = [48137,48138]
site_ids = set(sites_calib) | set(sites_nocalib)
print(site_ids)

Adding Datetime Timestamp and Delta T columns...
Data columns: Index(['site_id', 'pump_id', 'timestamp', 'facility_name', 'vibration',
       'thrust temperature', 'suction pressure', 'discharge pressure',
       'flow rate', 'frequency', 'amps', 'ampsa', 'ampsb', 'ampsc', 'volts',
       'voltsa', 'voltsb', 'voltsc', 'meter total', 'timestamp_datetime',
       'delta_t'],
      dtype='object')
Number of rows: 261571
Earliest timestamp: 2024-11-22 12:55:10
Latest timestamp: 2025-02-20 12:53:04
{33404, 48137, 48138, 33467, 57740, 33614}


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/preprocessing.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[frequency_int_column] = df[frequency_float_column].round().astype('Int64')


In [2]:
kpis_all_sites = {site_id:None for site_id in site_ids}
site_pump_curves = load_pump_curves(site_ids)
for site_id in site_ids: # for each site...
    site_data = deepcopy(data[data['site_id']==site_id]) # Looks at individual site data
    site_pump_kpis = None
    if site_id in sites_calib:
        calibration_stages = sites_info[site_id]['calibration_stages']
        calib_data = pd.DataFrame(columns=site_data.columns)
        for stage in calibration_stages:
            calib_data = pd.concat([calib_data,select_calib_data(site_data, stage, approx_time=True)])
        _, site_pump_kpis, _ = compute_kpis_for_sites(deepcopy(calib_data), site_id)
    else:
        _, site_pump_kpis, _ = compute_kpis_for_sites(site_data, site_id)
    site_name = sites_info[site_id]['site_name']
    kpis_all_sites[site_id] = site_pump_kpis

/var/folders/hf/ptl8z0cj3d3f1dwq2815fnnr0000gn/T/ipykernel_23239/2653512106.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  calib_data = pd.concat([calib_data,select_calib_data(site_data, stage, approx_time=True)])
/var/folders/hf/ptl8z0cj3d3f1dwq2815fnnr0000gn/T/ipykernel_23239/2653512106.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  calib_data = pd.concat([calib_data,select_calib_data(site_data, stage, approx_time=True)])
/var/folders/hf/ptl8z0cj3d3f1dwq2815fnnr0000gn/T/ipykernel_2

In [3]:
df_all, game_kpi = kpi_charts(kpis_all_sites, site_ids, sites_info, kpis=['kWh/BBL','Flow Rate'],print_chart=False)

df_site_48137 = pd.DataFrame(kpis_all_sites[48137]).transpose()
df_site_48138 = pd.DataFrame(kpis_all_sites[48138]).transpose()

common_rows = df_site_48137.index.intersection(df_site_48138.index)
common_cols = df_site_48138.columns.intersection(df_site_48137.columns)
df1_common = df_site_48137.loc[common_rows, common_cols]
df2_common = df_site_48138.loc[common_rows, common_cols]
comb_small_pumps = df1_common.add(df2_common,fill_value=0)
comb_small_pumps['kWh/BBL'] = comb_small_pumps['kWh/BBL'].div(2)

comb_small_pumps['site_id'] = 99999

df_all = pd.concat([df_all,comb_small_pumps])

fig1 = px.scatter(df_all,x='Flow Rate',y='kWh/BBL',color='site_id',height=700)
fig1.show()
fig2 = px.line(df_all,x='Flow Rate',y='kWh/BBL',color='site_id',height=700)
fig2.show()

Union City 2 SWD
1509 1 SWD
1509 2 SWD
Siegrist SWD
Canadian SWD
Calumet SWD


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/format_calculations.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all,df_site])
